In [4]:
import pandas as pd
import numpy as np
from scipy import sparse
from sklearn.preprocessing import normalize


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


ModuleNotFoundError: No module named 'pandas'

In [ ]:
groups = pd.read_csv('C:\LTC\groups.csv')
users = pd.read_csv('C:\LTC\sers.csv')
attends = pd.read_csv('C:\LTC\end.csv')

In [ ]:
prepared_attends = attends.drop(columns=['уникальный номер занятия', 'уникальный номер группы', 'направление 2', 'онлайн/офлайн', 'дата занятия', 'время начала занятия', 'время окончания занятия'])

In [ ]:
prepared_attends = prepared_attends.rename(columns={'уникальный номер участника':'user_id', 'направление 3':'lesson'})

In [ ]:
prepared_attends.index = prepared_attends.user_id

In [ ]:
at = attends.drop(columns=['уникальный номер занятия', 'уникальный номер группы', 'направление 2', 'дата занятия', 'время начала занятия', 'время окончания занятия'])

In [ ]:
at = at.rename(columns={'уникальный номер участника':'user_id', 'направление 3':'lesson'})

In [ ]:
at = at.groupby(['user_id', 'lesson'], as_index=False).agg({'онлайн/офлайн':'count'})

In [ ]:
at = at.rename(columns={'онлайн/офлайн':'quantity'})

In [ ]:
nums = [i + 1 for i in range(len(at.lesson.unique()))]

In [ ]:
lessons_dict = dict(zip(at.lesson.unique(), nums))

In [ ]:
prep_at = at.replace({'lesson':lessons_dict})

In [ ]:
title_dict = dict(zip(nums, at.lesson.unique()))

In [ ]:
rows, r_pos = np.unique(prep_at.values[:, 0], return_inverse=True)

In [ ]:
columns, c_pos = np.unique(prep_at.values[:, 1], return_inverse=True)

In [ ]:
int_sparse = sparse.csr_matrix((prep_at.values[:, 2], (r_pos, c_pos)))

In [ ]:
int_sparse

<50906x453 sparse matrix of type '<class 'numpy.int64'>'
	with 286056 stored elements in Compressed Sparse Row format>

In [ ]:
pui = normalize(int_sparse, norm='l2', axis=1)
sim = pui.T * pui

In [ ]:
int_sparse_T = int_sparse.transpose(copy=True)

In [ ]:
Piu = normalize(int_sparse_T, norm='l1', axis=1)
fit = pui * Piu * pui

In [ ]:
fit

<50906x453 sparse matrix of type '<class 'numpy.float64'>'
	with 21163322 stored elements in Compressed Sparse Row format>

In [ ]:
def get_rec(us_id, us_list, fit_mx, sparse_mx, lessons_dct):#Функция для получения рекомендаций
    ind = 0
    for i in range(len(us_list)):
        if us_list[i] == us_id:
            ind = i
            break
    initial_set = set([lessons_dct[i + 1] for i in np.nonzero(sparse_mx[ind])[1].tolist()])
    pred_set = set([lessons_dct[i + 1] for i in fit_mx[ind].toarray().argsort()[0][-30:].tolist()])
    return list(pred_set - initial_set)[:10]

In [ ]:
get_rec(10145659, rows, fit, int_sparse, title_dict)

['ОНЛАЙН Осваиваем мобильные устройства',
 'ОНЛАЙН Иные интеллектуальные игры',
 'ОНЛАЙН Психологические тренинги',
 'ОНЛАЙН Здорово жить',
 'ОНЛАЙН Мастер-класс по уходу за кожей в зрелом возрасте',
 'ОНЛАЙН Краеведение и онлайн-экскурсии',
 'ОНЛАЙН Дыхательная гимнастика',
 'Краеведение и пешие прогулки',
 'Скандинавская ходьба',
 'ОНЛАЙН Различные техники рисования']

In [ ]:
np.save('C:\PY\mx_sparse', int_sparse, allow_pickle=True)

In [ ]:
arr = np.load('C:\PY\lo_l.npy', allow_pickle=True)

In [ ]:
arr

array(<50906x453 sparse matrix of type '<class 'numpy.float64'>'
	with 21163322 stored elements in Compressed Sparse Row format>,
      dtype=object)

In [ ]:
def load_mx(filepath):
    return np.load(filepath, allow_pickle=True)

In [ ]:
at

,user_id,lesson,quantity
0,101346549,Авторские курсы/маршруты,1
1,101346549,Акварельная живопись,1
2,101346549,Здоровая спина,1
3,101346549,Литература,1
4,101346549,ОНЛАЙН История искусства,1
...,...,...,...
286051,101449498,Вязание на спицах,3
286052,101449549,ОНЛАЙН Адаптивная и тонизирующая гимнастика,2
286053,101449549,ОНЛАЙН Гимнастика мозга,1
286054,101449549,ОНЛАЙН Дизайн интерьера,1


In [ ]:
prep_at

,user_id,lesson,quantity
0,101346549,1,1
1,101346549,2,1
2,101346549,3,1
3,101346549,4,1
4,101346549,5,1
...,...,...,...
286051,101449498,219,3
286052,101449549,158,2
286053,101449549,104,1
286054,101449549,112,1


In [ ]:
at.to_csv('C:\PY\d.csv')

In [ ]:
prep_at

,user_id,lesson,quantity
0,101346549,1,1
1,101346549,2,1
2,101346549,3,1
3,101346549,4,1
4,101346549,5,1
...,...,...,...
286051,101449498,219,3
286052,101449549,158,2
286053,101449549,104,1
286054,101449549,112,1


In [ ]:
fit = np.load('C:\PY\mx_fit.npy', allow_pickle=True)

In [ ]:
int_sparse = np.load('C:\PY\mx_sparse.npy', allow_pickle=True)

In [ ]:
fit

array(<50906x453 sparse matrix of type '<class 'numpy.float64'>'
	with 21163322 stored elements in Compressed Sparse Row format>,
      dtype=object)

In [ ]:
int_sparse

array(<50906x453 sparse matrix of type '<class 'numpy.int64'>'
	with 286056 stored elements in Compressed Sparse Row format>,
      dtype=object)

In [ ]:
fit.tolist

<function ndarray.tolist>

In [ ]:
fit

array(<50906x453 sparse matrix of type '<class 'numpy.float64'>'
	with 21163322 stored elements in Compressed Sparse Row format>,
      dtype=object)

In [ ]:
fit1 = fit.tolist

In [ ]:
c = np.savetxt('C:\PY\iit.txt', fit, delimiter=', ')

ValueError: Expected 1D or 2D array, got 0D array instead

In [ ]:
fit1 = np.array(fit)

In [ ]:
prep_at

,user_id,lesson,quantity
0,101346549,1,1
1,101346549,2,1
2,101346549,3,1
3,101346549,4,1
4,101346549,5,1
...,...,...,...
286051,101449498,219,3
286052,101449549,158,2
286053,101449549,104,1
286054,101449549,112,1


In [ ]:
pd.read_csv('C:\PY\d.csv')

,Unnamed: 0,user_id,lesson,quantity
0,0,101346549,Авторские курсы/маршруты,1
1,1,101346549,Акварельная живопись,1
2,2,101346549,Здоровая спина,1
3,3,101346549,Литература,1
4,4,101346549,ОНЛАЙН История искусства,1
...,...,...,...,...
286051,286051,101449498,Вязание на спицах,3
286052,286052,101449549,ОНЛАЙН Адаптивная и тонизирующая гимнастика,2
286053,286053,101449549,ОНЛАЙН Гимнастика мозга,1
286054,286054,101449549,ОНЛАЙН Дизайн интерьера,1
